## Imports

In [1]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

## Code

In [2]:
import easyocr
import cv2
import os
from google.colab.patches import cv2_imshow

reader = easyocr.Reader(['en', 'hi'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import easyocr
import torch

def convert_to_fp16(model):
    """
    Converts a given model to use FP16.
    """
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            module.half()

# Load EasyOCR model
reader = easyocr.Reader(['hi'], gpu=True)  # Use 'gpu=True' to utilize GPU if available

# Access the detection and recognition models within EasyOCR
detector = reader.detector
recognizer = reader.recognizer

# Convert the detection and recognition models to FP16
convert_to_fp16(detector)
convert_to_fp16(recognizer)

In [16]:
from PIL import Image
import numpy as np

def convert_image_for_model(image_path):
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image).astype(np.float32) / 255.0  # Convert to float32
    return image_np


path = '/content/drive/MyDrive/OCR-SKU/Rough2/Hindi Images/'
image_path = path + os.listdir(path)[3]
image_np = convert_image_for_model(image_path)

# Perform OCR
result = reader.readtext(image_np, detail=0)
print(result)

RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [17]:
def load_custom_recognizer(model_path, device):
    """
    Load a custom recognizer model.
    """
    model = torch.load(model_path, map_location=device)
    model.eval()  # Set to evaluation mode
    return model

# Load EasyOCR with the default model
reader = easyocr.Reader(['hi'], gpu=True)  # Use 'gpu=True' to utilize GPU if available

# Load the custom recognizer model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
custom_recognizer_model_path = '/content/drive/MyDrive/OCR-SKU/Easy OCR/devanagari.pth'
custom_recognizer = load_custom_recognizer(custom_recognizer_model_path, device)

# Replace the default recognizer model with the custom one
reader.recognizer = custom_recognizer

# Ensure input images are correctly preprocessed
def preprocess_input(image_path):
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image).astype(np.float32) / 255.0  # Convert to float32 and normalize
    return image_np

image_np = preprocess_input(image_path)

# Perform OCR
result = reader.readtext(image_np, detail=0)
print(result)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
path = '/content/drive/MyDrive/OCR-SKU/Rough2/Hindi Images/'
#for image_names in sorted(os.listdir(path))
for i in range(len(os.listdir(path))):
    image_path = path + sorted(os.listdir(path))[i]
    img = cv2.imread(image_path)

    result = reader.readtext(img)

    entire_text = ''
    for bbox, text, score in result:
        entire_text += (text+' ')

    if 'शिश' in entire_text or 'शिशु' in entire_text or 'दुग्ध' in entire_text or 'आहार' in entire_text:
        print('AMULSPARY INFANT MILK FOOD INFANT MILK SUBSTITUTE')

    if 'अमूल्य' in entire_text or 'डेरी' in entire_text or 'व्हाइटनर' in entire_text or 'ब्हाइटनर' in entire_text:
        print('AMULYA DAIRY WHITENER')
    #print()
    cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# amulspray: अमूलख्रे अमूलख्प्रे अमूलर्प्रे ( शिश शिशु दुग्ध आहार)
# अमूल्य डेरी व्हाइटनर / डेरी ब्हाइटनर